In [1]:

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jainr\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from sklearn.model_selection import train_test_split
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

# Preprocessing

In [4]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents2.json').read() # you can find the intents file here:https://github.com/Sanket758/AI-Chatbot-with-Tensorflow
intents = json.loads(data_file)

In [5]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jainr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

52 documents
22 classes ['Appointment_Booking', 'Cardiovascular_risk_factors_discussion', 'Diabetes_risk_factors', 'Economic_burden_discussion', 'Emergency_Assistance', 'Explain_hypertension', 'Genetics_role_chronic_diseases', 'Greet', 'Health_Condition_Information', 'Health_Tips', 'Healthy_Lifestyle_Advice', 'Hypertension_medications', 'Importance_blood_pressure_monitoring', 'Importance_early_detection', 'Lifestyle_modifications_chronic_diseases', 'Medical_Facility_Locator', 'Public_health_concerns_statistics', 'Role_nutrition_diabetes_management', 'Symptoms_heart_diseases', 'Thyroid_disorders_explanation', 'Thyroid_disorders_symptoms', 'no-response']
137 unique lemmatized words ["'s", "'ve", '.', 'a', 'about', 'affect', 'any', 'appointment', 'are', 'aspect', 'been', 'benefit', 'better', 'bitten', 'blood', 'bp', 'by', 'can', 'cardiovascular', 'care', 'center', 'change', 'check', 'chronic', 'common', 'commonly', 'condition', 'contributing', 'control', 'controlling', 'cost', 'data', 'de

In [8]:

training = []

for document in documents:
    bag = []
    word_pattern = document[0]
    word_pattern = [lemmatizer.lemmatize(word.lower()) for word in word_pattern]
    for word in words:
        bag.append(1) if word in word_pattern else bag.append(0)

    output_row = [0] * len(classes)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)

X = np.array([item[0] for item in training])
y = np.array([item[1] for item in training])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [11]:
# Compile model with SGD optimizer
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


In [12]:
#fitting and saving the model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0468 - loss: 3.1467      
Epoch 2/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0728 - loss: 3.1102     
Epoch 3/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.0654 - loss: 3.0652      
Epoch 4/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1679 - loss: 3.0111 
Epoch 5/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1753 - loss: 2.9581 
Epoch 6/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2939 - loss: 2.8347 
Epoch 7/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.2787 - loss: 2.7120  
Epoch 8/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2064 - loss: 2.7111 
Epoch 9/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5037 - loss: 2.3982 
Epoch 10/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.4008 - loss: 2.5087  
Epoch 11/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3459 - loss: 2.3160 
Epoch 12/200
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

model created


In [13]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [14]:
intents = json.loads(open('intents2.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [15]:
# Utility Methods
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [16]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [17]:
print("bot is running")
while True:
    message = input("")
    ints = predict_class(message,model)
    res = getResponse(ints, intents)
    print(res)


bot is running
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Hypertension, or high blood pressure, occurs when the force of blood against the artery walls is consistently too high.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sorry, I didn't understand you.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Please go on.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [ ]:
import gradio.inputs as gi
print(dir(gi))


ModuleNotFoundError: No module named 'gradio.inputs'